# Predicción de tópicos
Se desea extraer los tópicos principales de cada semana dentro del ámbito político

## Librerias

In [15]:
import pandas as pd
import re # expresiones regulares (regex)
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer

# Preprocesamiento

In [2]:
# Archivo npr.csv es para hacer pruebas

df = pd.read_csv('noticias-economicas-sample.csv')

textoPrueba = df[['text']].copy() # Pongo .copy() porque cuando intentas 
                                  # quitar las stopwords sale un warning.
textoPrueba = textoPrueba.head(1000)

print(textoPrueba.shape)
textoPrueba.head(10)

(1000, 1)


,text
0,Angela Merkel ayer en Berlin tras la cumbre te...
1,\n\n\n\n\nEnrique Quemada Clariana\nFusiones y...
2,¿Cuántas veces no ha querido ser su propio jef...
3,"El ministro de Sanidad, Salvador Illa, momento..."
4,El Pleno del Congreso ha aprobado el proyecto ...
5,Vacunarse de la gripe para aliviar los hospita...
6,"Elon Musk, fundador de Tesla. EFE\n\nLa compañ..."
7,Una usuaria consulta la aplicación de banca ar...
8,Las consecuencias socioeconómicas de la pandem...
9,La aerolínea Ryanair anunció este viernes que ...


Eliminamos stopwords

In [3]:
# Función para eliminar stopwords en español

def noStopwords(text):
    
    sw_es = set(stopwords.words('spanish'))

    text = ' '.join([word for word in text.split() if word.lower() not in sw_es])
    
    return text


In [4]:
textoPrueba['text'] = textoPrueba['text'].apply(noStopwords)
textoPrueba.head()

,text
0,Angela Merkel ayer Berlin tras cumbre telemáti...
1,Enrique Quemada Clariana Fusiones adquisicione...
2,¿Cuántas veces querido ser propio jefe? ¿Cuánt...
3,"ministro Sanidad, Salvador Illa, momentos rued..."
4,Pleno Congreso aprobado proyecto Presupuestos ...


Normalizamos el texto, quitamos mayúsculas y signos de puntuación

In [5]:
# Función para normalizar el texto

def noPuntuacion(text): return re.findall(r'[a-zA-ZñÑ]+', text.lower()) # áéíóúÁÉÍÓÚüÜ

In [6]:
textoPrueba['text'] = textoPrueba['text'].apply(noPuntuacion).apply(lambda x: " ".join(x))
textoPrueba.head()

,text
0,angela merkel ayer berlin tras cumbre telem ti...
1,enrique quemada clariana fusiones adquisicione...
2,cu ntas veces querido ser propio jefe cu ntas ...
3,ministro sanidad salvador illa momentos rueda ...
4,pleno congreso aprobado proyecto presupuestos ...


Aplicamos lematización

In [7]:
# Función para lematizar al texto
def lemmatize_text(text):

    # Cargamos el módulo de español
    nlp = spacy.load("es_core_news_sm")

    # nlp crea una pipeline que le adhiere metadata que la paquetería spacy usa
    doc = nlp(text)

    lemmatized_text = " ".join([token.lemma_ for token in doc])
    
    return lemmatized_text

In [8]:
textoPrueba['text'] = textoPrueba['text'].apply(lemmatize_text)
textoPrueba.head()

,text
0,angelar merkel ayer berlin tras cumbre telem t...
1,enrique quemado clarián fusión adquisición act...
2,cu ntar vez querido ser propio jefe cu nta vez...
3,ministro sanidad salvador illa momento rodar p...
4,pleno congreso aprobado proyecto presupuesto g...


Aplicamos stemming (Esta madre lo mas probable es que no lo pongamos)

In [106]:
# Función para aplicar stemming al texto

def stemmiar(text):

    stemmer = SnowballStemmer('spanish')

    return ' '.join([stemmer.stem(word) for word in text.split()])

In [107]:
textoPrueba['text'] = textoPrueba['text'].apply(stemmiar)
textoPrueba.head()

,text
0,angel merkel ayer berlin tras cumbr telem tic ...
1,enriqu quem clarian fusion adquisicion actual ...
2,cu ntas vec quer ser propi jef cu ntas vec arr...
3,ministr sanid salvador illa moment rued prens ...
4,plen congres aprob proyect presupuest general ...


Vectorizamos y hacemos n-gramas

In [11]:
# Creamos una instancia que hará la vectorización TF-IDF
vect = TfidfVectorizer()

# Tokenizamos como Penn Treebank
tokenizer = TreebankWordTokenizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# Incluimos 1-grams y 2-grams
vect.set_params(ngram_range=(1, 2))

# Ignoramos términos que aparecen en más del 50% de los documentos
vect.set_params(max_df=0.5)

# Solo mantenemos los términos que aparecen en al menos 2 documentos
vect.set_params(min_df=2)

TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2),
                tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000001434674B0A0>>)

In [14]:
# Vectorizamos
vect_fit = vect.fit_transform(textoPrueba['text']) 
vect_fit

<1000x40639 sparse matrix of type '<class 'numpy.float64'>'
	with 301393 stored elements in Compressed Sparse Row format>